In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl.drawing.image import Image
from io import BytesIO
from math import sin, cos, exp, sqrt, pi

In [2]:
# Evaluation functions
def f1(x):
    return -5 / (1 + np.sum(x ** 2)) + sin(1 / np.tan(exp(-5 / (1 + np.sum(x ** 2)))))

def f2(x, a=20, b=0.2, c=2 * pi):
    n = len(x)
    sum_sq_term = -a * exp(-b * sqrt(np.sum(x ** 2) / n))
    cos_term = -exp(np.sum(np.cos(c * x)) / n)
    return sum_sq_term + cos_term + a + exp(1)

In [3]:
def binary_to_real(binary_individual, domain, dimensions):
    max_value = 2 ** 16 - 1  # Max val for 16 bits
    real_individual = []
    for i in range(dimensions):
        binary_value = int("".join(map(str, binary_individual[i*16:(i+1)*16])), 2)
        real_value = domain[0] + (binary_value / max_value) * (domain[1] - domain[0])
        real_individual.append(real_value)
    return np.array(real_individual)

In [4]:
def initialize_population(pop_size, dimensions, domain, binary=False):
    if binary:
        return np.random.randint(2, size=(pop_size, dimensions * 16))  # Binary
    return np.random.uniform(domain[0], domain[1], (pop_size, dimensions)) # Decimal

In [5]:
def tournament_selection(pop, scores, k=3):
    selected = np.random.choice(range(len(pop)), k)
    best = selected[np.argmin([scores[i] for i in selected])]
    return pop[best]

In [6]:
def crossover(parent1, parent2, binary=False):
    if binary:
        point = np.random.randint(1, len(parent1) - 1)
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
    else:
        alpha = np.random.rand()
        child1 = alpha * parent1 + (1 - alpha) * parent2
        child2 = alpha * parent2 + (1 - alpha) * parent1
    return child1, child2

In [7]:
def mutate(individual, mutation_rate=0.01, binary=False, domain=(-5, 5)):
    if binary:
        for i in range(len(individual)):
            if np.random.rand() < mutation_rate:
                individual[i] = 1 - individual[i]
    else:
        individual += np.random.normal(0, 0.1, size=individual.shape)
        individual = np.clip(individual, domain[0], domain[1])
    return individual

In [8]:
def evolutionary_algorithm(pop_size, dimensions, domain, generations, eval_func, binary=False, max_evaluations=10000):
    pop = initialize_population(pop_size, dimensions, domain, binary)
    scores = []
    best_scores = []
    eval_count = 0

    while eval_count < max_evaluations:
        # Decoding binary
        if binary:
            decoded_pop = [binary_to_real(ind, domain, dimensions) for ind in pop]
            scores = np.array([eval_func(ind) for ind in decoded_pop])
        else:
            scores = np.array([eval_func(ind) for ind in pop])

        best = np.min(scores)
        best_scores.append(best)
        eval_count += len(scores)

        # Creating new population
        new_pop = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(pop, scores)
            parent2 = tournament_selection(pop, scores)
            child1, child2 = crossover(parent1, parent2, binary)
            new_pop.append(mutate(child1, binary=binary, domain=domain))
            new_pop.append(mutate(child2, binary=binary, domain=domain))
        pop = np.array(new_pop)

    return best_scores

In [9]:
def run_experiments():
    results = {}
    results['F1_real'] = [evolutionary_algorithm(pop_size, dimensions, domain_f1, generations, f1, binary=False) for _ in range(100)]
    results['F2_real'] = [evolutionary_algorithm(pop_size, dimensions, domain_f2, generations, f2, binary=False) for _ in range(100)]
    results['F1_binary'] = [evolutionary_algorithm(pop_size, dimensions, domain_f1, generations, f1, binary=True) for _ in range(100)]
    results['F2_binary'] = [evolutionary_algorithm(pop_size, dimensions, domain_f2, generations, f2, binary=True) for _ in range(100)]
    return results

In [10]:
def save_results_to_excel(results):
    with pd.ExcelWriter("results.xlsx", engine="openpyxl") as writer:
        for name, data in results.items():
            data = np.array(data)
            df_raw = pd.DataFrame(data)
            df_raw.to_excel(writer, sheet_name=f'{name}_raw', index=False)

            mean_data = np.mean(data, axis=0)
            df_mean = pd.DataFrame(mean_data, columns=['Mean'])
            df_mean.to_excel(writer, sheet_name=f'{name}_mean', index=False)

            plt.figure()
            plt.plot(mean_data, label=f'Average Best Score ({name})')
            plt.xlabel("Generation")
            plt.ylabel("Fitness")
            plt.legend()
            plt.title(f'Average Best Scores - {name}')

            img_data = BytesIO()
            plt.savefig(img_data, format='png')
            img_data.seek(0)
            plt.close()

            workbook = writer.book
            worksheet = workbook[f'{name}_mean']
            img = Image(img_data)
            worksheet.add_image(img, 'E5')

In [11]:
# Experiment variables and run
pop_size = 50
dimensions = 10
domain_f1 = (-3, 3)
domain_f2 = (-32.768, 32.768)
generations = 200

In [12]:
results = run_experiments()

In [13]:
save_results_to_excel(results)